In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import keras
from keras import layers

1. Chargement du dataset

In [22]:
csv_path = "iris.csv"

df = pd.read_csv(csv_path, sep=r"\s+")

print("Colonnes du CSV :", df.columns.tolist())
print(df.head())

feature_cols = ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]
label_col = "Species"

X = df[feature_cols].values
y = df[label_col].values

print("Shape X :", X.shape)
print("Espèces :", np.unique(y))

Colonnes du CSV : ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']
   Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
Shape X : (150, 4)
Espèces : ['setosa' 'versicolor' 'virginica']


2. Standardisation + ACP brute

In [23]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca_raw = PCA(n_components=2)
X_pca_raw = pca_raw.fit_transform(X_scaled)

species = np.unique(y)
color_map = {
    species[0]: "red",
    species[1]: "green",
    species[2]: "blue",
}

3. Encodage des labels + split train/val

In [ ]:
n_samples, n_features = X_scaled.shape
assert n_features == 4

sp_to_idx = {
    "setosa": 0,
    "versicolor": 1,
    "virginica": 2,
}

idx_to_sp = {i: sp for sp, i in sp_to_idx.items()}

y_int = np.array([sp_to_idx[sp] for sp in y], dtype="int32")

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_int, test_size=0.2, random_state=42, stratify=y_int
)

print("X_train shape :", X_train.shape)
print("X_val shape   :", X_val.shape)

X_train shape : (120, 4)
X_val shape   : (30, 4)


4. Modele self-attention + Dense pour classification

In [ ]:
n_features = 4

inputs = keras.Input(shape=(n_features,),name="iris_inputs")

x_seq = layers.Reshape((n_features, 1), name="to_sequence")(inputs)

att = layers.Attention(name="self_attention")([x_seq, x_seq])

att_pooled = layers.GlobalAveragePooling1D(name="att_pool")(att)

latent = layers.Dense(16, activation="relu", name="latent_dense")(att_pooled)

outputs = layers.Dense(3, activation="softmax")(latent)

model = keras.Model(inputs=inputs, outputs=outputs, name="iris_attention_classifier")
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    batch_size=16,
    epochs=100,
    callbacks=callbacks,
    verbose=1,
)

Model: "iris_attention_classifier"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ iris_inputs         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ to_sequence         │ (None, 4, 1)      │          0 │ iris_inputs[0][0] │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention      │ (None, 4, 1)      │          0 │ to_sequence[0][0… │
│ (Attention)         │                   │            │ to_sequence[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_pool            │ (None, 1)         │          0 │ self_attention[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_dense        │ (None, 16)        │         32 │ att_pool[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │         51 │ latent_dense[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 83 (332.00 B)

 Trainable params: 83 (332.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0583 - loss: 1.1049 - val_accuracy: 0.3333 - val_loss: 1.1075
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3417 - loss: 1.0813 - val_accuracy: 0.3333 - val_loss: 1.0882
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3667 - loss: 1.0590 - val_accuracy: 0.4000 - val_loss: 1.0695
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4750 - loss: 1.0385 - val_accuracy: 0.6667 - val_loss: 1.0514
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6833 - loss: 1.0189 - val_accuracy: 0.6667 - val_loss: 1.0345
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6833 - loss: 1.0001 - val_accuracy: 0.6667 - val_loss: 1.0176
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6833 - loss: 0.9820 - val_accuracy: 0.6667 - val_loss: 1.0015
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6833 - loss: 0.9640 - val_accuracy: 0.6667 - val_loss: 0.9863

5. Extraction des representations

In [ ]:
att_model = keras.Model(
    inputs=model.input,
    outputs=model.get_layer("self_attention").output,
    name="encoder_attention_only"
)

encoder_latent = keras.Model(
    inputs=model.input,
    outputs=model.get_layer("latent_dense").output,
    name="encoder_attention_plus_dense",
)

H_att_raw = att_model.predict(X_scaled, batch_size=32)
H_att = H_att_raw.reshape(H_att_raw.shape[0], -1)

H_lat = encoder_latent.predict(X_scaled, batch_size=32)

H_att_scaled = StandardScaler().fit_transform(H_att)
H_lat_scaled = StandardScaler().fit_transform(H_lat)

pca_att = PCA(n_components=2)
H_att_pca = pca_att.fit_transform(H_att_scaled)

pca_lat = PCA(n_components=2)
H_lat_pca = pca_lat.fit_transform(H_lat_scaled)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


6. Affichage des resultats

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
species_sorted = sorted(np.unique(y))

# Panel 1 : ACP brute
ax = axes[0]
for sp in species_sorted:
    mask = (y == sp)
    ax.scatter(
        X_pca_raw[mask, 0],
        X_pca_raw[mask, 1],
        label=sp,
        alpha=0.7,
        s=30,
        edgecolor="k",
    )
ax.set_title("ACP brute (dataset d'origine)")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.legend()
ax.grid(True)

# Panel 2 : ACP self-attention standalone
ax = axes[1]
for sp in species_sorted:
    mask = (y == sp)
    ax.scatter(
        H_att_pca[mask, 0],
        H_att_pca[mask, 1],
        label=sp,
        alpha=0.7,
        s=30,
        edgecolor="k",
    )
ax.set_title("ACP self-attention standalone")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.legend()
ax.grid(True)

# Panel 3 : ACP self-attention + Dense
ax = axes[2]
for sp in species_sorted:
    mask = (y == sp)
    ax.scatter(
        H_lat_pca[mask, 0],
        H_lat_pca[mask, 1],
        label=sp,
        alpha=0.7,
        s=30,
        edgecolor="k",
    )
ax.set_title("ACP self-attention + Dense")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.savefig("iris_pca_three_panels.png", dpi=150)
plt.close()